<a href="https://colab.research.google.com/github/123prashanth123/Fault-Detection-System/blob/Colabs/Feature%20Patch%20Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random as r

import torch
from torch import nn
from torchvision import models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from time import time
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

def breaker(num=50, char="*"):
    print("\n" + num*char + "\n")

patch_h, patch_w = 8, 8

# Build Feature Extractor


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.size = 224
        self.transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model = models.resnet50(pretrained=True, progress=True)
        self.model = nn.Sequential(*[*self.model.children()][:-1])
        self.model.add_module("Flatten", nn.Flatten())
    
    def normalize(self, x):
        for i in range(x.shape[0]):
            x[i] = (x[i] - torch.min(x[i])) / (torch.max(x[i]) - torch.min(x[i]))
        return x

    def get_features(self, image):
        self.to(self.device)
        image = cv2.resize(src=image, dsize=(self.size, self.size), interpolation=cv2.INTER_AREA)
        with torch.no_grad():
            features = self(self.transform(image).to(self.device).unsqueeze(dim=0))
        return self.normalize(features).detach().cpu(), self.normalize(features).detach().cpu().numpy()

    def forward(self, x):
        return self.model(x)


model = Model()
model.eval()

clear_output()

# Process Patch-by-Patch

In [ ]:
name = "M6.jpg"
image = cv2.resize(src=cv2.cvtColor(src=cv2.imread("/content/" + name, cv2.IMREAD_COLOR), code=cv2.COLOR_BGR2RGB), dsize=(640, 360), interpolation=cv2.INTER_AREA)

h, w, _ = image.shape

if h % patch_h == 0:num_cols = int(h/patch_h)
else:num_cols = int(h/patch_h) + 1
if w % patch_w == 0:num_rows = int(w/patch_w)
else:num_rows = int(w/patch_w) + 1

breaker()
print("Columns, Rows : {}, {}".format(num_cols, num_rows))
breaker()

image = cv2.resize(src=image, dsize=(num_rows*patch_w, num_cols*patch_h), interpolation=cv2.INTER_AREA)

patches = []
for i in range(0, h, patch_h):
    for j in range(0, w, patch_w):
        patches.append([i, patch_h+i, j, patch_w+j])
patches = np.array(patches).reshape(num_cols, num_rows, 4)

torch_features, numpy_features = [], []
start_time = time()
for i in range(num_cols):
    for j in range(num_rows):
        image_patch = image[patches[i][j][0]:patches[i][j][1], patches[i][j][2]:patches[i][j][3], :]
        torch_features.append(model.get_features(image_patch)[0])
        numpy_features.append(model.get_features(image_patch)[1])
# torch_features = torch.FloatTensor(torch_features)
# numpy_features = np.array(numpy_features)
print("Time taken for {} Patches : {:.2f} seconds".format(num_cols * num_rows, time() - start_time))
breaker()


**************************************************

Columns, Rows : 45, 80

**************************************************

Time taken for 3600 Patches : 56.73 seconds

**************************************************



# Batch Patch Processing

In [ ]:
name = "M6.jpg"
image = cv2.resize(src=cv2.cvtColor(src=cv2.imread("/content/" + name, cv2.IMREAD_COLOR), code=cv2.COLOR_BGR2RGB), dsize=(640, 360), interpolation=cv2.INTER_AREA)

h, w, _ = image.shape

if h % patch_h == 0:num_cols = int(h/patch_h)
else:num_cols = int(h/patch_h) + 1
if w % patch_w == 0:num_rows = int(w/patch_w)
else:num_rows = int(w/patch_w) + 1

breaker()
print("Columns, Rows : {}, {}".format(num_cols, num_rows))
breaker()

image = cv2.resize(src=image, dsize=(num_rows*patch_w, num_cols*patch_h), interpolation=cv2.INTER_AREA)

patches = []
for i in range(0, h, patch_h):
    for j in range(0, w, patch_w):
        patches.append([i, patch_h+i, j, patch_w+j])
patches = np.array(patches).reshape(num_cols, num_rows, 4)

image_patches = []
start_time = time()
for i in range(num_cols):
    for j in range(num_rows):
        image_patches.append(image[patches[i][j][0]:patches[i][j][1], patches[i][j][2]:patches[i][j][3], :])
print("Time taken for {} Patches : {:.2f} seconds".format(num_cols * num_rows, time() - start_time))
breaker()

class DS(Dataset):
    def __init__(self, X=None, transform=None):
        self.transform = transform
        self.X = X

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.transform(self.X[idx])


data_setup = DS(X=image_patches, transform=model.transform)
data = DL(data_setup, batch_size=64, shuffle=False)
features = torch.empty((0, 2048), device=model.device)

start_time = time()
for X in data:
    X = X.to(model.device)
    with torch.no_grad():
        output = model(X)
    features = torch.cat((features, output), dim=0)


print("Time taken for {} Patches : {:.2f} seconds".format(num_cols * num_rows, time() - start_time))
breaker()


**************************************************

Columns, Rows : 45, 80

**************************************************

Time taken for 3600 Patches : 0.01 seconds

**************************************************

Time taken for 3600 Patches : 0.68 seconds

**************************************************

